In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import h5py
import numpy as np

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 20,
})

script_dir = os.getcwd()
# ---------- here is the path to the results directory ----------------------
# here is the path to the results directory
# ---------------------------------------------------------------------------
results_dir_path = Path(f"{script_dir}/outputs/qbp/exact_validation")
dir_per_size = os.walk(results_dir_path).__next__()[1]
errors = []
sizes = []
def trace_distance(lhs, rhs):
    s = np.linalg.eigvalsh(lhs - rhs)
    return 0.5 * np.abs(s).sum(-1).mean()
for size in dir_per_size:
    error_per_size = []
    size_path = results_dir_path.joinpath(size)
    for subpath in os.walk(size_path).__next__()[1]:
        result_path = size_path.joinpath(subpath)
        with h5py.File(result_path.joinpath("result"), "r") as f:
            density_matrices = np.array(f["density_matrices_history"])
        with h5py.File(result_path.joinpath("exact_result"), "r") as f:
            exact_density_matrices = np.array(f["density_matrices_history"])
        error_per_size.append(trace_distance(exact_density_matrices, density_matrices))
    sizes.append(int(size))
    errors.append(np.array(error_per_size))
errors = np.array(errors)
sizes = np.array(sizes)
errors = errors[sizes.argsort()][..., np.newaxis]
sizes.sort()
sizes = np.tile(sizes.reshape((-1, 1, 1)), (1, errors.shape[1], 1))
data = np.concatenate([sizes, errors], axis=-1).reshape((-1, 2))
fig = plt.figure()
plt.scatter(data[:, 0], data[:, 1], marker="d", color="b")
plt.ylabel(r"${\epsilon}$")
plt.xlabel(r"${\rm Qubits \ number}$")
plt.xticks(sizes[:, 0, 0])
plt.yscale('log')

In [ ]:
fig.savefig(f"{results_dir_path}/error_analysis.pdf",  bbox_inches="tight")